In [6]:
import torch
import numpy as np
import sys

sys.path.append('../human_body_prior/tools/')
from omni_tools import copy2cpu as c2c
from os import path as osp

both57m_dir = '../data/both57m/'
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(comp_device)

sys.path.append('../human_body_prior/body_model/')
from body_model import BodyModel

gender = "male"

bm_fname = '../smplh_models/{}/model.npz'.format(gender)
bm = BodyModel(bm_fname=bm_fname).to(comp_device)
faces = c2c(bm.f)

cuda


In [7]:
import trimesh
sys.path.append('../body_visualizer')
from tools.vis_tools import colors
from mesh.mesh_viewer import MeshViewer
from mesh.sphere import points_to_spheres
from tools.vis_tools import show_image

imw, imh=1600, 1600
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

data_number = 500

both57m_npz = osp.join(both57m_dir, str(data_number), 'data.npz')
bdata = np.load(both57m_npz)
len = bdata['seq_len']

motion_parms = {
    'root_orient': torch.Tensor(bdata['root_orient']).to(comp_device),
    'pose_body': torch.Tensor(bdata['body_pose'].reshape(len,-1)).to(comp_device),
    'pose_hand': torch.Tensor(bdata['hand_pose'].reshape(len,-1)).to(comp_device),
    'trans': torch.Tensor(bdata['trans']).to(comp_device)
}

body_pose = bm(**{k:v for k,v in motion_parms.items() if k in ['pose_body', 'pose_hand', 'root_orient']})
joints = np.zeros((len, 52, 3))

for i in range(len):
    array = c2c(body_pose.Jtr[i])
    array = array[:,[2,0,1]]
    joints[i] = array

print(type(joints), joints.shape)
np.save('joints.npy', joints)

<class 'numpy.ndarray'> (150, 52, 3)
